# Neo4j Hello World (Notebook) - SEC Use Case

This notebook connects to a local Neo4j **Community** instance (via Docker), creates a tiny graph, and queries it.

**Assumes** 
 
 
- Neo4j service is running at `bolt://localhost:${URI_PORT}` with the user and password set in the `.env` file. **Run `docker compose up -d`**.
- Ollama service is up on `http://localhost:11434` (ollama default). **Run `ollama serve` and pull the model `ollama pull nomic-embed-text`** (if not pulled yet).

In [1]:

# Dependencies

import os
from dotenv import load_dotenv  
import yaml
from pathlib import Path
from pprint import pprint
from termcolor import cprint
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter

# === Neo4j / LangChain
from langchain_google_vertexai import ChatVertexAI
from langchain_ollama import ChatOllama

# === Local services
from neo4j_service import Neo4jService



In [2]:
# Environment variables

load_dotenv()  # Load local environment variables
MODEL_SERVER = os.getenv("MODEL_SERVER")
MODEL_NAME = os.getenv("MODEL_NAME")

In [3]:
# Neo4j Langchain wrapper instance
kg = Neo4jService.get_graph()

Initializing graph at bolt://localhost:7687


## 1+2. Create data with rich text (chunks)

In [4]:
# Load data from file

file_name = "./data/SEC/form10k.json" # form10k for the Netapp company

# LangChain Text splitter for chunking process
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)

def split_form10k_data_from_file(file):
    
    chunks_with_metadata = [] # accumlate chunk records
    
    data = json.load(open(file)) # open the json file
    for item in ['item1','item1a','item7','item7a']: # pull these keys from the json
        
        print(f'Processing {item} from {file}') 
        
        item_text_chunks = text_splitter.split_text(data[item]) # split the text into chunks
        
        chunk_seq_id = 0
        for chunk in item_text_chunks: # only take the first 20 chunks
            
            form_id = file[file.rindex('/') + 1:file.rindex('.')] # extract form id from file name
            
            # finally, construct a record with metadata and the chunk text
            chunks_with_metadata.append({
                'text': chunk, 
                'f10kItem': item,
                'chunkSeqId': chunk_seq_id,
                # constructed metadata...
                'formId': f'{form_id}', # pulled from the filename
                'uuid': f'{form_id}-{item}-chunk{chunk_seq_id:04d}',
                # metadata from file...
                'names': data['names'],
                'cik': data['cik'],
                'cusip6': data['cusip6'],
                'source': data['source'],
            })
            
            chunk_seq_id += 1
            
        print(f'\t{item} splitted into {chunk_seq_id} chunks')
        
    return chunks_with_metadata


chunks_dicts = split_form10k_data_from_file(file_name)

Processing item1 from ./data/SEC/form10k.json
	item1 splitted into 254 chunks
Processing item1a from ./data/SEC/form10k.json
	item1a splitted into 1 chunks
Processing item7 from ./data/SEC/form10k.json
	item7 splitted into 1 chunks
Processing item7a from ./data/SEC/form10k.json
	item7a splitted into 1 chunks


In [5]:
# Populate graph
   
dbinfo = kg.query("CALL db.info()")
cprint(f"\nConnected to Neo4j database: {dbinfo[0]['name']}", "green")

cprint("\nCreating constraints (if not exist)", "green")
constraints = ["""
CREATE CONSTRAINT chunk_unique IF NOT EXISTS 
    FOR (c:Chunk) REQUIRE c.uuid IS UNIQUE
"""]
for q in constraints:
    kg.query(q)

qs = [
    """MATCH (n) DETACH DELETE n
    """,
    """DROP INDEX chunks_node_idx IF EXISTS
    """
]
cprint("\nInit Cleanup.", "green")
for q in qs:
    kg.query(q)

cprint("\nCreate data", "green")
node_count = 0
for chunk_dict in chunks_dicts:
    print(f"Creating `:Chunk` node for chunk ID {chunk_dict['uuid']}")
    
    
    query = """
    MERGE(c:Chunk {uuid: $chunkParamDict.uuid})
    ON CREATE SET 
        c.names = $chunkParamDict.names,
        c.formId = $chunkParamDict.formId, 
        c.cik = $chunkParamDict.cik, 
        c.cusip6 = $chunkParamDict.cusip6, 
        c.source = $chunkParamDict.source, 
        c.f10kItem = $chunkParamDict.f10kItem, 
        c.chunkSeqId = $chunkParamDict.chunkSeqId, 
        c.text = $chunkParamDict.text
    RETURN c
    """
    kg.query(query, 
    params={
        'chunkParamDict': chunk_dict
        }
    )
    node_count += 1
    
print(f"Created {node_count} nodes")

2025-09-30 14:07:54,208 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintDoesNotExist} {category: SCHEMA} {title: `DROP INDEX chunks_node_idx IF EXISTS` has no effect.} {description: `chunks_node_idx` does not exist.} {position: None} for query: 'DROP INDEX chunks_node_idx IF EXISTS\n    '



Connected to Neo4j database: neo4j

Creating constraints (if not exist)

Init Cleanup.

Create data
Creating `:Chunk` node for chunk ID form10k-item1-chunk0000
Creating `:Chunk` node for chunk ID form10k-item1-chunk0001
Creating `:Chunk` node for chunk ID form10k-item1-chunk0002
Creating `:Chunk` node for chunk ID form10k-item1-chunk0003
Creating `:Chunk` node for chunk ID form10k-item1-chunk0004
Creating `:Chunk` node for chunk ID form10k-item1-chunk0005
Creating `:Chunk` node for chunk ID form10k-item1-chunk0006
Creating `:Chunk` node for chunk ID form10k-item1-chunk0007
Creating `:Chunk` node for chunk ID form10k-item1-chunk0008
Creating `:Chunk` node for chunk ID form10k-item1-chunk0009
Creating `:Chunk` node for chunk ID form10k-item1-chunk0010
Creating `:Chunk` node for chunk ID form10k-item1-chunk0011
Creating `:Chunk` node for chunk ID form10k-item1-chunk0012
Creating `:Chunk` node for chunk ID form10k-item1-chunk0013
Creating `:Chunk` node for chunk ID form10k-item1-chunk0014

In [6]:
# Create conections and form nodes

    
# Create a node to represent the entire Form 10-K
# Get form metadata from any chunk
query = """
MATCH (anyChunk:Chunk) 
     WITH anyChunk LIMIT 1
    RETURN 
        anyChunk.names as names, 
        anyChunk.source as source, 
        anyChunk.formId as formId, 
        anyChunk.cik as cik, 
        anyChunk.cusip6 as cusip6
"""
form_info_list = kg.query(query)

if not form_info_list:
    print("No chunks found in the database")
else:
    form_record = form_info_list[0]
    print("Form info retrieved:")
    pprint(form_record)
    
    # Create the Form node with individual parameters
    cypher = """
        MERGE (f:Form {formId: $formId})
            ON CREATE 
            SET f.names = $names,
                f.source = $source,
                f.cik = $cik,
                f.cusip6 = $cusip6
    """
    
    query = """
    MERGE (f:Form {formId: $formId})
    ON CREATE 
    SET 
        f.names = $names,
        f.source = $source,
        f.cik = $cik,
        f.cusip6 = $cusip6

    """
    # Pass individual parameters instead of nested dictionary
    kg.query(query, {
        'formId': form_record['formId'],
        'names': form_record['names'],
        'source': form_record['source'],
        'cik': form_record['cik'],
        'cusip6': form_record['cusip6']
    })
    
        
    # Verify the Form node was created, Show the created Form node details
    qs = ["""
          MATCH (f:Form) RETURN count(f) as formCount
          ""","""
          MATCH (f:Form) RETURN f
          """]
    for q in qs:
        result = kg.query(q)
        for r in result:
            pprint(dict(r))
        
    # Create a linked list of Chunk nodes for each section
    query = """
    MATCH (c:Chunk)
        WHERE c.formId = $formId
        AND c.f10kItem = $f10kItem
    WITH c
        ORDER BY c.chunkSeqId ASC
    WITH collect(c) as section_chunk_list
    CALL apoc.nodes.link(
        section_chunk_list, 
        "NEXT", 
        {avoidDuplicates: true}
    )  // NEW!!!
    RETURN size(section_chunk_list)
    """
    for form10kItemName in ['item1', 'item1a', 'item7', 'item7a']:
        kg.query(query, {'formId': form_record['formId'],
                                    'f10kItem': form10kItemName})
        
        
    query = """
    MATCH (c:Chunk), (f:Form)
        WHERE c.formId = f.formId
    MERGE (c)-[newRelationship:PART_OF]->(f)
    RETURN count(newRelationship)
    """
    # Connect chunks to their parent form with a PART_OF relationship
    kg.query(query)
    
    
    query = """
    MATCH (first:Chunk), (f:Form)
    WHERE first.formId = f.formId
        AND first.chunkSeqId = 0
    WITH first, f
        MERGE (f)-[r:SECTION {f10kItem: first.f10kItem}]->(first)
    RETURN count(r)
    """
    # Create a SECTION relationship on first chunk of each section
    kg.query(query)
    

Form info retrieved:
{'cik': '1002047',
 'cusip6': '64110D',
 'formId': 'form10k',
 'names': ['Netapp Inc', 'NETAPP INC'],
 'source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}
{'formCount': 1}
{'f': {'cik': '1002047',
       'cusip6': '64110D',
       'formId': 'form10k',
       'names': ['Netapp Inc', 'NETAPP INC'],
       'source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}}


2025-09-30 14:07:55,920 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {category: PERFORMANCE} {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (f))} {position: line: 2, column: 5, offset: 5} for query: '\n    MATCH (c:Chunk), (f:Form)\n        WHERE c.formId = f.formId\n    MERGE (c)-[newRelationship:PART_OF]->(f)\n    RETURN count(newRelationship)\n    '
2025-09-30 14:07:55,995 - neo4j.notifications - INFO - Received notific

In [7]:
# Example cypher queries



# Return the first chunk of the Item 1 section

query = """
MATCH (f:Form)-[r:SECTION]->(first:Chunk)
      WHERE f.formId = $formId
          AND r.f10kItem = $f10kItem
    RETURN first.uuid as uuid, first.text as text
"""
result = kg.query(query, 
                      {'formId': form_record['formId'],
                      'f10kItem': 'item1'})

first_chunk_info = dict(list(result)[0])
print(first_chunk_info)

# Get the second chunk of the Item 1 section
query = """
MATCH (first:Chunk)-[:NEXT]->(nextChunk:Chunk)
      WHERE first.uuid = $uuid
    RETURN nextChunk.uuid as uuid, nextChunk.text as text
"""
result = kg.query(query, 
                      {'uuid': first_chunk_info['uuid']})

next_chunk_info = dict(list(result)[0])

print(next_chunk_info)

# See relationships between form node and the first and second chunks of each section. Try it out in browser!!
query = """
MATCH (n:Chunk)-[r]-(f:Form)
    WHERE n.chunkSeqId IN [0, 1]
RETURN n.text, type(r) AS relType, f.formId
"""
result = kg.query(query, {'uuid': first_chunk_info['uuid']})
for r in result:
  print(r)
  
# Return a window of three chunks

result = kg.query("""
                  MATCH (c1:Chunk)-[:NEXT]->(c2:Chunk)-[:NEXT]->(c3:Chunk) 
                  WHERE c2.uuid = $uuid
                  RETURN c1.uuid, c2.uuid, c3.uuid
                  """,
                  {'uuid': next_chunk_info['uuid']})

for r in result:
    print(r)
    
result = kg.query("""
                  MATCH window=
                  (:Chunk)-[:NEXT*0..1]->(c:Chunk)-[:NEXT*0..1]->(:Chunk) 
                  WHERE c.uuid = $uuid
                  WITH window as longestChunkWindow 
                  ORDER BY length(longestChunkWindow) DESC
                  RETURN length(longestChunkWindow) 
                  """,
                  {'uuid': first_chunk_info['uuid']})

for r in result:
    print(r)
  


{'uuid': 'form10k-item1-chunk0000', 'text': '>Item 1.  \nBusiness\n\n\nOverview\n\n\nNetApp, Inc. (NetApp, we, us or the Company) is a global cloud-led, data-centric software company. We were incorporated in 1992 and are headquartered in San Jose, California. Building on more than three decades of innovation, we give customers the freedom to manage applications and data across hybrid multicloud environments. Our portfolio of cloud services, and storage infrastructure, powered by intelligent data management software, enables applications to run faster, more reliably, and more securely, all at a lower cost.\n\n\nOur opportunity is defined by the durable megatrends of data-driven digital and cloud transformations. NetApp helps organizations meet the complexities created by rapid data and cloud growth, multi-cloud management, and the adoption of next-generation technologies, such as AI, Kubernetes, and modern databases. Our modern approach to hybrid, multicloud infrastructure and data mana

## 3. Create property embeddings (first step into RAG) 

In [8]:
# Create vector index
query = """
CREATE VECTOR INDEX `chunks_node_idx` IF NOT EXISTS
FOR (c:Chunk) ON (c.embedding) 
OPTIONS { indexConfig: {`vector.dimensions`: 768, `vector.similarity_function`: 'cosine'}}
"""
kg.query(query)

# Show created vector indexes
results = kg.query("SHOW VECTOR INDEXES")
idx = results
cprint(f"\nFound {len(idx)} vector index entries.", "green")
for r in idx:
    cprint("-"*20,"green")
    pprint(r)



Found 4 vector index entries.
--------------------
{'entityType': 'NODE',
 'id': 5,
 'indexProvider': 'vector-2.0',
 'labelsOrTypes': ['Chunk'],
 'lastRead': None,
 'name': 'chunks_node_idx',
 'owningConstraint': None,
 'populationPercent': 100.0,
 'properties': ['embedding'],
 'readCount': None,
 'state': 'ONLINE',
 'type': 'VECTOR'}
--------------------
{'entityType': 'NODE',
 'id': 8,
 'indexProvider': 'vector-2.0',
 'labelsOrTypes': ['Company'],
 'lastRead': neo4j.time.DateTime(2025, 9, 30, 12, 6, 0, 483000000, tzinfo=<UTC>),
 'name': 'company_node_idx',
 'owningConstraint': None,
 'populationPercent': 100.0,
 'properties': ['embedding'],
 'readCount': 1,
 'state': 'ONLINE',
 'type': 'VECTOR'}
--------------------
{'entityType': 'RELATIONSHIP',
 'id': 9,
 'indexProvider': 'vector-2.0',
 'labelsOrTypes': ['KNOWS'],
 'lastRead': neo4j.time.DateTime(2025, 9, 30, 12, 6, 2, 242000000, tzinfo=<UTC>),
 'name': 'know_relationship_idx',
 'owningConstraint': None,
 'populationPercent': 100.

In [9]:
# Create property embeddings 
    
Neo4jService.vectorize_property(
                    element = "node",
                    node_label = "Chunk",
                    source_property = "text"
                    )

2025-09-30 14:07:56,545 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"



Generating embeddings for (n:Chunk) on n.text
 Updated 1 embeddings


2025-09-30 14:07:56,624 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


 Updated 2 embeddings


2025-09-30 14:07:56,674 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:07:56,725 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:07:56,783 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:07:56,813 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


...

 Updated 241 embeddings
 Updated 242 embeddings
 Updated 243 embeddings
 Updated 244 embeddings
 Updated 245 embeddings
 Updated 246 embeddings
 Updated 247 embeddings
 Updated 248 embeddings


2025-09-30 14:08:03,852 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:08:03,879 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:08:03,906 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:08:03,935 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:08:03,961 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:08:03,987 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:08:04,020 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:08:04,049 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


 Updated 249 embeddings
 Updated 250 embeddings
 Updated 251 embeddings
 Updated 252 embeddings
 Updated 253 embeddings
 Updated 254 embeddings
 Updated 255 embeddings
 Updated 256 embeddings


2025-09-30 14:08:04,079 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


 Updated 257 embeddings


## 4. Search 

In [11]:
# KG RAG Search

# Query Nodes
result = Neo4jService.neo4j_KGRAG_search(
                            query = 'In a single sentence, tell me about Netapp.',
                            index = 'chunks_node_idx',
                            source_property = "text",
                            main_property = "uuid",
                            top_k = 10
                            )

pprint(result, width = 200, sort_dicts=False, indent=2)
file = "data/SEC/SEC_context.txt"
with open(file, 'w', encoding='utf-8') as f:
  f.write(result.get("combined_context",""))


2025-09-30 14:09:22,361 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"



Running vector search query to retrieve context.
{ 'query': 'In a single sentence, tell me about Netapp.',
  'total_results': 10,
  'raw_search_results': [ { 'score': 0.8418002128601074,
                            'label': ['Chunk'],
                            'properties_dict': { 'cik': '1002047',
                                                 'text': '•\n'
                                                         'NetApp Keystone is our pay-as-you-grow, storage-as-a-service (STaaS) offering that delivers a seamless hybrid cloud experience for those '
                                                         'preferring operating expense consumption models to upfront capital expense or leasing. With a unified management console and monthly bill '
                                                         'for both on-premises and cloud data storage services, Keystone lets organizations provision and monitor, and even move storage spend across '
                                       

**Create conections:**

Chunks belong to Forms, Chunks follow other Chunks and some of them are the head of section of the Form.

<p align="center">
  <img src="media/KGRAG_SEC_example.png">
</p>

<p align="center">
  <img src="media/KGRAG_SEC_example_2.png">
</p>



In [12]:
# Add investment information about what firms have invested in which companies (NetApp in our case is what matters)

import csv

all_form13s = []

with open('./data/SEC/form13.csv', mode = 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for r in csv_reader: # each row is a dictionary
        all_form13s.append(r)
        
len(all_form13s)

561

In [13]:
# For each investment we create a manager and a company node.

first_form13 = all_form13s[0]
cypher = """
MERGE (com:Company {cusip6: $cusip6})
    ON CREATE
        SET com.companyName = $companyName,
            com.cusip = $cusip
"""


kg.query(cypher,
            {'cusip6':first_form13['cusip6'],
                        'companyName':first_form13['companyName'],
                        'cusip':first_form13['cusip']
                        })

[]

In [14]:
# For now, there is only one company - NetApp
cypher = """
MATCH (com:Company)
RETURN com LIMIT 1
"""

pprint(kg.query(cypher)[0])

{'com': {'companyName': 'NETAPP INC', 'cusip': '64110D104', 'cusip6': '64110D'}}


In [15]:
# Update the company name to match Form 10-K

cypher = """
MATCH (com:Company), (form:Form)
WHERE com.cusip6 = form.cusip6
RETURN com.companyName, form.names
"""


pprint(kg.query(cypher)[0])

cypher = """
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  SET com.names = form.names
"""


print(kg.query(cypher))

2025-09-30 14:09:22,627 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {category: PERFORMANCE} {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (form))} {position: line: 2, column: 1, offset: 1} for query: '\nMATCH (com:Company), (form:Form)\nWHERE com.cusip6 = form.cusip6\nRETURN com.companyName, form.names\n'
2025-09-30 14:09:22,657 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION

{'com.companyName': 'NETAPP INC', 'form.names': ['Netapp Inc', 'NETAPP INC']}
[]


In [16]:
# Create a FILED relationship between the company and the Form-10K node
cypher ="""
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  MERGE (com)-[:FILED]->(form)
"""

print(kg.query(cypher))

2025-09-30 14:09:22,703 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {category: PERFORMANCE} {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (form))} {position: line: 2, column: 3, offset: 3} for query: '\n  MATCH (com:Company), (form:Form)\n    WHERE com.cusip6 = form.cusip6\n  MERGE (com)-[:FILED]->(form)\n'


[]


In [17]:
# Create manager nodes for companies that have filed a Form 13 to report their investment in NetApp
# Start with the single manager who filed the first Form 13 in the list

cypher = """
MERGE (mgr:Manager {uuid: $uuid})
ON CREATE
    SET mgr.managerName = $managerName,
        mgr.managerAddress = $managerAddress
"""

params = {
    "uuid": first_form13["managerCik"],
    "managerName": first_form13["managerName"],
    "managerAddress": first_form13["managerAddress"]
}

print(kg.query(cypher, params))


[]


In [18]:
cypher = """
  MATCH (mgr:Manager)
  RETURN mgr LIMIT 1
"""


pprint(kg.query(cypher)[0])

{'mgr': {'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, '
                           'M5J2J5',
         'managerName': 'Royal Bank of Canada',
         'uuid': '1000275'}}


In [19]:
# Create a uniquness constraint to avoid duplicate managers
cypher ="""
  CREATE CONSTRAINT manager_unique 
  IF NOT EXISTS
  FOR (n:Manager) 
  REQUIRE n.uuid IS UNIQUE
"""

print(kg.query(cypher))

[]


In [20]:
# Create a fulltext index of manager names to enable text search

cypher ="""
CREATE FULLTEXT INDEX fullTextManagerNames
  IF NOT EXISTS
  FOR (mgr:Manager) 
  ON EACH [mgr.managerName]
"""

print(kg.query(cypher))

[]


In [21]:
cypher ="""
CALL db.index.fulltext.queryNodes("fullTextManagerNames", 
      "royal bank") YIELD node, score
  RETURN node.managerName, score
"""

print(kg.query(cypher))

[{'node.managerName': 'Royal Bank of Canada', 'score': 0.2615291476249695}]


In [22]:
# Create nodes for all companies that filed a Form 13
cypher = """
  MERGE (mgr:Manager {uuid: $uuid})
    ON CREATE
        SET mgr.managerName = $managerName,
            mgr.managerAddress = $managerAddress
"""
for form13 in all_form13s:
  params = {
      "uuid": form13["managerCik"],
      "managerName": form13["managerName"],
      "managerAddress": form13["managerAddress"]
  }

  
print(kg.query(cypher, params))


# cypher = """
#   MERGE (mgr:Manager {uuid: $managerParam.managerCik})
#     ON CREATE
#         SET mgr.managerName = $managerParam.managerName,
#             mgr.managerAddress = $managerParam.managerAddress
# """
# # loop through all Form 13s
# for form13 in all_form13s:
#   kg.query(cypher, params={'managerParam': form13 })

[]


In [23]:
cypher = """
    MATCH (mgr:Manager) 
    RETURN count(mgr)
"""


pprint(kg.query(cypher)[0])

{'count(mgr)': 2}


In [24]:
# Create relationships between managers and companies
cypher = """
  MATCH (mgr:Manager {uuid: $investmentParam.managerCik}), 
        (com:Company {cusip6: $investmentParam.cusip6})
  RETURN mgr.managerName, com.companyName, $investmentParam as investment
"""

kg.query(cypher, params={ 
    'investmentParam': first_form13 
})

2025-09-30 14:09:23,025 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {category: PERFORMANCE} {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (com))} {position: line: 2, column: 3, offset: 3} for query: '\n  MATCH (mgr:Manager {uuid: $investmentParam.managerCik}), \n        (com:Company {cusip6: $investmentParam.cusip6})\n  RETURN mgr.managerName, com.companyName, $investmentParam as investment\n'


[{'mgr.managerName': 'Royal Bank of Canada',
  'com.companyName': 'NETAPP INC',
  'investment': {'shares': '842850',
   'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
   'managerName': 'Royal Bank of Canada',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'value': '64395000000.0',
   'cusip6': '64110D',
   'cusip': '64110D104',
   'reportCalendarOrQuarter': '2023-06-30',
   'companyName': 'NETAPP INC',
   'managerCik': '1000275'}}]

In [25]:
cypher = """
MATCH (mgr:Manager {uuid: $ownsParam.managerCik}), 
        (com:Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN { 
    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter
}]->(com)
ON CREATE
    SET owns.value  = toFloat($ownsParam.value), 
        owns.shares = toInteger($ownsParam.shares)
RETURN mgr.managerName, owns.reportCalendarOrQuarter, com.companyName
"""

kg.query(cypher, params={ 'ownsParam': first_form13 })

2025-09-30 14:09:23,106 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {category: PERFORMANCE} {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (com))} {position: line: 2, column: 1, offset: 1} for query: '\nMATCH (mgr:Manager {uuid: $ownsParam.managerCik}), \n        (com:Company {cusip6: $ownsParam.cusip6})\nMERGE (mgr)-[owns:OWNS_STOCK_IN { \n    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter\n}]->(com)\nON CRE

[{'mgr.managerName': 'Royal Bank of Canada',
  'owns.reportCalendarOrQuarter': '2023-06-30',
  'com.companyName': 'NETAPP INC'}]

In [26]:
kg.query("""
MATCH (mgr:Manager {uuid: $ownsParam.managerCik})
-[owns:OWNS_STOCK_IN]->
        (com:Company {cusip6: $ownsParam.cusip6})
RETURN owns { .shares, .value }
""", params={ 'ownsParam': first_form13 })

[{'owns': {'shares': 842850, 'value': 64395000000.0}}]

In [27]:
# Create relationships between all of the managers who filed Form 13s and the company
cypher = """
MATCH (mgr:Manager {uuid: $ownsParam.managerCik}), 
        (com:Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN { 
    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter 
    }]->(com)
  ON CREATE
    SET owns.value  = toFloat($ownsParam.value), 
        owns.shares = toInteger($ownsParam.shares)
"""

#loop through all Form 13s
for form13 in all_form13s:
  kg.query(cypher, params={'ownsParam': form13 })

2025-09-30 14:09:23,234 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {category: PERFORMANCE} {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (com))} {position: line: 2, column: 1, offset: 1} for query: '\nMATCH (mgr:Manager {uuid: $ownsParam.managerCik}), \n        (com:Company {cusip6: $ownsParam.cusip6})\nMERGE (mgr)-[owns:OWNS_STOCK_IN { \n    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter \n    }]->(com)\n 

In [28]:
cypher = """
  MATCH (:Manager)-[owns:OWNS_STOCK_IN]->(:Company)
  RETURN count(owns) as investments
"""

kg.query(cypher)

[{'investments': 2}]

In [29]:
kg.refresh_schema()
print(kg.schema, 60)

Node properties:
Company {cusip: STRING, names: LIST, cusip6: STRING, companyName: STRING}
Chunk {names: LIST, formId: STRING, embedding: LIST, uuid: STRING, text: STRING, source: STRING, f10kItem: STRING, chunkSeqId: INTEGER, cik: STRING, cusip6: STRING}
Form {source: STRING, names: LIST, formId: STRING, cik: STRING, cusip6: STRING}
Manager {uuid: STRING, managerName: STRING, managerAddress: STRING}
Relationship properties:
SECTION {f10kItem: STRING}
OWNS_STOCK_IN {shares: INTEGER, reportCalendarOrQuarter: STRING, value: FLOAT}
The relationships:
(:Company)-[:FILED]->(:Form)
(:Chunk)-[:PART_OF]->(:Form)
(:Chunk)-[:NEXT]->(:Chunk)
(:Form)-[:SECTION]->(:Chunk)
(:Manager)-[:OWNS_STOCK_IN]->(:Company) 60


In [30]:
# Determine the number of investors
cypher = """
    MATCH (chunk:Chunk)
    RETURN chunk.uuid as uuid LIMIT 1
    """

chunk_rows = kg.query(cypher)
print(chunk_rows)
chunk_first_row = chunk_rows[0]
print(chunk_first_row)
ref_chunk_id = chunk_first_row['uuid']
print(ref_chunk_id)

[{'uuid': 'form10k-item1-chunk0040'}]
{'uuid': 'form10k-item1-chunk0040'}
form10k-item1-chunk0040


In [31]:
# Build up path from Form 10-K chunk to companies and managers
cypher = """
    MATCH (:Chunk {uuid: $chunkIdParam})-[:PART_OF]->(f:Form)
    RETURN f.source
    """

kg.query(cypher, params={'chunkIdParam': ref_chunk_id})

[{'f.source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}]

In [32]:
cypher = """
MATCH (:Chunk {uuid: $chunkIdParam})-[:PART_OF]->(f:Form),
    (com:Company)-[:FILED]->(f)
RETURN com.companyName as name
"""

kg.query(cypher, params={'chunkIdParam': ref_chunk_id})

[{'name': 'NETAPP INC'}]

In [33]:
# Use queries to build additional context for LLM
cypher = """
MATCH (:Chunk {uuid: $chunkIdParam})-[:PART_OF]->(f:Form),
        (com:Company)-[:FILED]->(f),
        (mgr:Manager)-[:OWNS_STOCK_IN]->(com)
RETURN com.companyName, 
        count(mgr.managerName) as numberOfinvestors 
LIMIT 1
"""

kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})

[{'com.companyName': 'NETAPP INC', 'numberOfinvestors': 2}]

In [34]:
cypher = """
    MATCH (:Chunk {uuid: $chunkIdParam})-[:PART_OF]->(f:Form),
        (com:Company)-[:FILED]->(f),
        (mgr:Manager)-[owns:OWNS_STOCK_IN]->(com)
    RETURN mgr.managerName + " owns " + owns.shares + 
        " shares of " + com.companyName + 
        " at a value of $" + 
        apoc.number.format(toInteger(owns.value)) AS text
    LIMIT 10
    """
kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})

[{'text': 'BOKF, NA owns 40774 shares of NETAPP INC at a value of $3,115,134,000'},
 {'text': 'Royal Bank of Canada owns 842850 shares of NETAPP INC at a value of $64,395,000,000'}]

In [35]:
results = kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})
print(results[0]['text'], 60)

BOKF, NA owns 40774 shares of NETAPP INC at a value of $3,115,134,000 60


In [36]:
# # Create a plain Question Answer chain: Similarity search only, no augmentation by Cypher Query

# from langchain_community.vectorstores import Neo4jVector
# from langchain.chains import RetrievalQAWithSourcesChain
# from langchain_ollama import ChatOllama
# from langchain_ollama import OllamaEmbeddings


# vector_store = Neo4jVector.from_existing_graph(
#     embedding=OllamaEmbeddings(model='nomic-embed-text'),
#     url=URI,
#     username=NEO4J_USER,
#     password=NEO4J_PWD,
#     index_name="chunks_node_idx",
#     node_label="Chunk",
#     text_node_properties=["text"],
#     embedding_node_property="embedding",
# )
# # Create a retriever from the vector store
# retriever = vector_store.as_retriever()

# # Create a chatbot Question & Answer chain from the retriever
# plain_chain = RetrievalQAWithSourcesChain.from_chain_type(
#     ChatOllama(model='qwen3:8b',temperature=0, reasoning = False), 
#     chain_type="stuff", 
#     retriever=retriever
# )

In [37]:
# # Create a second QA chain: Augment similarity search using sentences found by the investment query above
# investment_retrieval_query = """
# MATCH (node)-[:PART_OF]->(f:Form),
#     (f)<-[:FILED]-(com:Company),
#     (com)<-[owns:OWNS_STOCK_IN]-(mgr:Manager)
# WITH node, score, mgr, owns, com 
#     ORDER BY owns.shares DESC LIMIT 10
# WITH collect (
#     mgr.managerName + 
#     " owns " + owns.shares + 
#     " shares in " + com.companyName + 
#     " at a value of $" + 
#     apoc.number.format(toInteger(owns.value)) + "." 
# ) AS investment_statements, node, score
# RETURN apoc.text.join(investment_statements, "\n") + 
#     "\n" + node.text AS text,
#     score,
#     { 
#       source: node.source
#     } as metadata
# """

# vector_store_with_investment = Neo4jVector.from_existing_index(
#     OllamaEmbeddings(model='nomic-embed-text'),
#     url=URI,
#     username=NEO4J_USER,
#     password=NEO4J_PWD,
#     database=NEO4J_DB,
#     index_name="chunks_node_idx",
#     text_node_property="text",
#     retrieval_query=investment_retrieval_query,
# )

# # Create a retriever from the vector store
# retriever_with_investments = vector_store_with_investment.as_retriever()

# # Create a chatbot Question & Answer chain from the retriever
# investment_chain = RetrievalQAWithSourcesChain.from_chain_type(
#     ChatOllama(model='qwen3:8b',temperature=0, reasoning = False), 
#     chain_type="stuff", 
#     retriever=retriever_with_investments
# )

In [38]:
# question = "In a single sentence, tell me about Netapp."
# plain_chain(
#     {"question": question},
#     return_only_outputs=True,
# )

In [39]:
# investment_chain(
#     {"question": question},
#     return_only_outputs=True,
# )

# Writing Cypher with an LLM

Use few-shot learning to teach an LLM to write Cypher
- You'll use the model ...
- You'll also use a new Neo4j integration within LangChain called **GraphCypherQAChain**

In [43]:
from langchain.prompts.prompt import PromptTemplate
from langchain_neo4j import GraphCypherQAChain
CYPHER_GENERATION_TEMPLATE_V1 = """Task:Generate Cypher statement to 
query a graph database.
Instructions:
Use only the provided relationship types and properties in the 
schema. Do not use any other relationship types or properties that 
are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher 
statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName
The question is:
{question}"""



CYPHER_GENERATION_TEMPLATE_V2 = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# What investment firms are near Santa Clara?
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress

The question is:
{question}"""


CYPHER_GENERATION_TEMPLATE_V3 = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# What investment firms are near Santa Clara?
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress

# What does Palo Alto Networks do?
  CALL db.index.fulltext.queryNodes(
         "fullTextCompanyNames", 
         "Palo Alto Networks"
         ) YIELD node, score
  WITH node as com
  MATCH (com)-[:FILED]->(f:Form),
    (f)-[s:SECTION]->(c:Chunk)
  WHERE s.f10kItem = "item1"
RETURN c.text

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE_V1
)

cprint(f"\nUsing LLM {MODEL_NAME} through VertexAI API.", "green")
if MODEL_SERVER == "OLLAMA":
  llm = ChatOllama(
        model=MODEL_NAME,
        temperature=0.2,
        num_ctx=16000,
        n_seq_max=1,
        extract_reasoning=False,
    )
  
if MODEL_SERVER == "VERTEX":
  
  llm = ChatVertexAI(
    model=MODEL_NAME,   # or "gemini-2.5-pro"
    temperature=0.2,
    max_output_tokens=1000,
    location="us-central1",     # or "europe-west1"
  )
  
cypherChain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    allow_dangerous_requests = True
    )

def prettyCypherChain(question: str) -> str:
    response = cypherChain.run(question)
    print(response, 60)
    
prettyCypherChain("What investment firms have invested more money?")


Using LLM gpt-oss:20b through VertexAI API.


> Entering new GraphCypherQAChain chain...


/tmp/ipykernel_2641809/1778953279.py:126: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = cypherChain.run(question)
2025-09-30 14:09:53,456 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Generated Cypher:
MATCH (m:Manager)-[o:OWNS_STOCK_IN]->(c:Company)
RETURN m.managerName, sum(o.value) AS totalInvestment
ORDER BY totalInvestment DESC;
Full Context:
[{'m.managerName': 'Royal Bank of Canada', 'totalInvestment': 64395000000.0}, {'m.managerName': 'BOKF, NA', 'totalInvestment': 3115134000.0}]


2025-09-30 14:09:55,759 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"



> Finished chain.
Royal Bank of Canada has invested more money. 60


In [44]:
kg.refresh_schema

<bound method Neo4jGraph.refresh_schema of <langchain_neo4j.graphs.neo4j_graph.Neo4jGraph object at 0x72bab81a5e80>>

In [45]:
kg.refresh_schema()
print(kg.schema, 60)

Node properties:
Company {cusip: STRING, names: LIST, cusip6: STRING, companyName: STRING}
Chunk {names: LIST, formId: STRING, embedding: LIST, uuid: STRING, text: STRING, source: STRING, f10kItem: STRING, chunkSeqId: INTEGER, cik: STRING, cusip6: STRING}
Form {source: STRING, names: LIST, formId: STRING, cik: STRING, cusip6: STRING}
Manager {uuid: STRING, managerName: STRING, managerAddress: STRING}
Relationship properties:
SECTION {f10kItem: STRING}
OWNS_STOCK_IN {shares: INTEGER, reportCalendarOrQuarter: STRING, value: FLOAT}
The relationships:
(:Company)-[:FILED]->(:Form)
(:Chunk)-[:PART_OF]->(:Form)
(:Chunk)-[:NEXT]->(:Chunk)
(:Form)-[:SECTION]->(:Chunk)
(:Manager)-[:OWNS_STOCK_IN]->(:Company) 60
